In [ ]:
#XML
#!/usr/bin/env python
# Your task here is to extract data from xml on authors of an article
# and add it to a list, one item for an author.
# See the provided data structure for the expected format.
# The tags for first name, surname and email should map directly
# to the dictionary keys
import xml.etree.ElementTree as ET

article_file = "exampleResearchArticle.xml"


def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()


def get_authors(root):
    authors = []
    for author in root.findall('./fm/bibl/aug/au'):
        #answer:
        # for i in insr:
        #     data["insr"].append(i.attrib["iid"])
        insr_group = []
        for insr in author.findall('insr'):
            insr_group.append(insr.get('iid'))
        data = {
                "fnm": author.find('fnm').text,
                "snm": author.find('snm').text,
                "email": author.find('email').text,
                "insr":insr_group
        }
        
            

        # YOUR CODE HERE

        authors.append(data)

    return authors


def test():
    solution = [{'fnm': 'Omer', 'snm': 'Mei-Dan', 'email': 'omer@extremegate.com'}, {'fnm': 'Mike', 'snm': 'Carmont', 'email': 'mcarmont@hotmail.com'}, {'fnm': 'Lior', 'snm': 'Laver', 'email': 'laver17@gmail.com'}, {'fnm': 'Meir', 'snm': 'Nyska', 'email': 'nyska@internet-zahav.net'}, {'fnm': 'Hagay', 'snm': 'Kammar', 'email': 'kammarh@gmail.com'}, {'fnm': 'Gideon', 'snm': 'Mann', 'email': 'gideon.mann.md@gmail.com'}, {'fnm': 'Barnaby', 'snm': 'Clarck', 'email': 'barns.nz@gmail.com'}, {'fnm': 'Eugene', 'snm': 'Kots', 'email': 'eukots@gmail.com'}]
    
    root = get_root(article_file)
    data = get_authors(root)

    assert data[0] == solution[0]
    assert data[1]["fnm"] == solution[1]["fnm"]


# test()
# get_authors(get_root(article_file))

In [ ]:
#USE BeautifulSoup
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Please note that the function 'make_request' is provided for your reference only.
# You will not be able to to actually use it from within the Udacity web UI.
# Your task is to process the HTML using BeautifulSoup, extract the hidden
# form field values for "__EVENTVALIDATION" and "__VIEWSTATE" and set the appropriate
# values in the data dictionary.
# All your changes should be in the 'extract_data' function
from bs4 import BeautifulSoup
import requests
import json

html_page = "page_source.html"


def extract_data(page):
    data = {"eventvalidation": "",
            "viewstate": ""}
    with open(page, "r") as html:
        # do something here to find the necessary values
        soup = BeautifulSoup(html, "lxml")
        data["eventvalidation"]  = soup.find(id="__EVENTVALIDATION")['value']
        data["viewstate"] = soup.find(id="__VIEWSTATE")['value']
        pass

    return data


def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]
    print 'befor request!!!!!'
    r = requests.post("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
                    data={'AirportList': "BOS",
                          'CarrierList': "VX",
                          'Submit': 'Submit',
                          "__EVENTTARGET": "",
                          "__EVENTARGUMENT": "",
                          "__EVENTVALIDATION": eventvalidation,
                          "__VIEWSTATE": viewstate
                    })
    print 'request Done!'
    return r.text


def test():
    data = extract_data(html_page)
    assert data["eventvalidation"] != ""
    assert data["eventvalidation"].startswith("/wEWjAkCoIj1ng0")
    assert data["viewstate"].startswith("/wEPDwUKLTI")

    
# test()
make_request(extract_data(html_page))

print make_request(extract_data(html_page))

In [ ]:
#beautifulSoup练习





from bs4 import BeautifulSoup
html_page = "page_source.html"


def extract_carriers(page):
    data = []

    with open(page, "r") as html:
        # do something here to find the necessary values
        soup = BeautifulSoup(html, "lxml")
        for option_value in  soup.find(id="CarrierList").find_all('option'):
            if option_value['value'].find('All') == -1:
                data.append(option_value['value']) 

    return data

def extract_airports(page):
    data = []
    with open(page, "r") as html:
        # do something here to find the necessary values
        soup = BeautifulSoup(html, "lxml")
        for option_value in  soup.find(id="AirportList").find_all('option'):
            if option_value['value'].find('All') == -1:
                data.append(option_value['value']) 
        

    return data

def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]
    airport = data["airport"]
    carrier = data["carrier"]
    print 'Before'
    r = s.post("https://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
               data = (("__EVENTTARGET", ""),
                       ("__EVENTARGUMENT", ""),
                       ("__VIEWSTATE", viewstate),
                       ("__VIEWSTATEGENERATOR",viewstategenerator),
                       ("__EVENTVALIDATION", eventvalidation),
                       ("CarrierList", carrier),
                       ("AirportList", airport),
                       ("Submit", "Submit")))

    return r.text


def test():
    data = extract_carriers(html_page)
    assert len(data) == 16
    assert "FL" in data
    assert "NK" in date
# if __name__ == "__main__":
#     test()
    
print extract_carriers(html_page)
print extract_airports(html_page)

In [ ]:
#假设你将上两个练习中的代码与关于如何构建请求的课程中代码相结合，并将所有数据下载到了本地。
#文件位于目录“data”下，按照航空公司和机场命名："{}-{}.html".format(carrier, airport)，例如“FL-ATL.html”。
#包含航班信息的表格具有一个表格类“dataTDRight”。你的任务是使用“process_file()”从该表格中提取航班数据，
#并作为字典列表，每个字典包含文件中的相关信息和表格行。以下是你应该返回的数据结构示例：
#Note - year, month, and the flight data should be integers.
#You should skip the rows that contain the TOTAL data for a year.
#注意：年月和航班数据应该是整型。你应该跳过包含一年 TOTAL 数据的行。
#你可以使用几个辅助函数来处理数据文件。为了方便打分，请勿更改这些辅助函数。你只需更改“process_file()”函数。
#上述标签中的“data/FL-ATL.html”文件只是完整数据的一部分，涵盖的是截止 2003 年的数据。
#test() 代码将使用完整的表格，但是给定文件应该提供一个你将获得的数据示例。
from bs4 import BeautifulSoup
from zipfile import ZipFile
import os

datadir = "data"


def open_zip(datadir):
    with ZipFile('{0}.zip'.format(datadir), 'r') as myzip:
        myzip.extractall()


def process_all(datadir):
    files = os.listdir(datadir)
    return files


def process_file(f):
    """
    This function extracts data from the file given as the function argument in
    a list of dictionaries. This is example of the data structure you should
    return:

    data = [{"courier": "FL",
             "airport": "ATL",
             "year": 2012,
             "month": 12,
             "flights": {"domestic": 100,
                         "international": 100}
            },
            {"courier": "..."}
    ]


    Note - year, month, and the flight data should be integers.
    You should skip the rows that contain the TOTAL data for a year.
    """
    data = []
    
    info = {}
    info["courier"], info["airport"] = f[:6].split("-")
    # Note: create a new dictionary for each entry in the output data list.
    # If you use the info dictionary defined here each element in the list 
    # will be a reference to the same info dictionary.
    with open("{}/{}".format(datadir, f), "r") as html:
        print 'start'

        
        soup = BeautifulSoup(html,"lxml")
        trs = soup.find_all("tr", { "class" : "dataTDRight" })
        entry = {}
        entry["courier"], info["airport"] = f[:6].split("-")
        
        # print  trs[0].find_all('td')[0].text
        for tr in trs:
            if tr.find_all('td')[1].text != 'TOTAL' :
             year = tr.find_all('td')[0].text
             month = tr.find_all('td')[1].text
             domestic = (tr.find_all('td')[2].text).replace(',','')
                
                
                
             international = (tr.find_all('td')[3].text).replace(',','')
                # print domestic
                # print international
             entry['year'] = int(year)
             entry['month'] = int(month)
             entry['flights'] = {'domestic':int(domestic),
                                   'international':int(international)}
                
             data.append(entry) 
            
            
            
                
            # else:
            #     info['year'] = tr.find_all('td')[0].text
            #     info['month'] = tr.find_all('td')[1].text
            #     info_in['domestic'] = tr.find_all('td')[2].text
            #     print info_in['domestic']
            #     info_in['international'] = tr.find_all('td')[3].text
            #     info['flights'] = info_in
               
         
           
            # if tr.find_all('td')[1].text == 'TOTAL':  
            #     print tr.find_all('td')[0].text
            #     print tr.find_all('td')[2].text
            #     print tr.find_all('td')[3].text
            #     print tr.find_all('td')[4].text
            # else:
            #     print tr.find_all('td')[0].text
            #     print tr.find_all('td')[1].text
            #     print tr.find_all('td')[2].text
            #     print tr.find_all('td')[3].text
               
            # 
       
       
        
    print 'end'
    
    return data


def test():
    print "Running a simple test..."
    open_zip(datadir)
    files = process_all(datadir)
    data = []
    # Test will loop over three data files.
    for f in files:
        data += process_file(f)
        
    assert len(data) == 399  # Total number of rows
    for entry in data[:3]:
        assert type(entry["year"]) == int
        assert type(entry["month"]) == int
        assert type(entry["flights"]["domestic"]) == int
        assert len(entry["airport"]) == 3
        assert len(entry["courier"]) == 2
    assert data[0]["courier"] == 'FL'
    assert data[0]["month"] == 10
    assert data[-1]["airport"] == "ATL"
    assert data[-1]["flights"] == {'international': 108289, 'domestic': 701425}
    
    print "... success!"

# if __name__ == "__main__":
#     test()

process_file('FL-ATL.html')

In [ ]:
import xml.etree.ElementTree as ET

PATENTS = 'patent.data'

def get_root(fname):

    tree = ET.parse(fname)
    return tree.getroot()


# get_root(PATENTS)

# 解答

# 很棒的尝试！问题出在：
# 不应该在遍历输入文件的每一行时，
# 就把输出文件名全部循环切换一遍哦。
# 
# 讲一下我自己的思路，供你参考：
# 
# 必需步骤：
# 
# 无论如何是需要打开并遍历 输入文件 的，这个逃不掉。
# 输出文件 是个变量。肯定需要循环下列切换操作：
# a. 打开输出文件
# b. 关闭输出文件
# c. 输出文件名加 1
# 难点在于 a, b, c 如何分组，以及何时进行切换操作。
# 
# 何时切换很容易解决，必须是 遇到 '<?xml' 时 进行。
# 但直接硬来会出现问题。
# 
# 初次（不正确的）尝试：
# 
# a. 打开
# b. 关闭
# c. +1
# 
# a. 打开
# b. 关闭
# c. +1
# 
# a. 打开
# b. 关闭
# c. +1
# 
# ……
# 每次遇到 '<?xml' 时这样分组切换的话，
# 虽然文件名有 +1，但打开后却直接关掉了，
# 导致 并没有机会使用输出文件 :joy:。
# 
# 那么如何解决呢？
# 
# 只要保持有输出文件处于打开状态就好了。
# 我们可以首先 单独打开一次输出文件，
# 然后每次遇到 '<?xml' 时，切换步骤变成 “关闭 、+1、再打开”：
# （总体顺序没有变，但是分组变化了）
# 
# a. 首次打开（循环之外单独进行）
# 
# b. 关闭
# c. +1
# a. 打开
# 
# b. 关闭
# c. +1
# a. 打开
# 
# b. 关闭
# c. +1
# a. 打开

# def split_file(filename):
#     with open(filename) as infile:
#         n = -1 # 由于第一次遇到 '<?xml' 时会关闭文件，导致跳空一次
#                # 所以临时从 -1 开始，保证能从 0 正式开始
#         outfile = open('{}-{}'.format(filename, n), 'w') # 单独打开
# 
#         for line in infile:
#             if line.startswith('<?xml'):
#                 # 每次切换步骤：关闭、+1、再打开
#                 outfile.close() 
#                 n += 1       
#                 outfile = open('{}-{}'.format(filename, n),'w')
# 
#             outfile.write(line)
#             
#         outfile.close() # 记得用完关闭


def split_file(filename):
    """
    Split the input file into separate files, each containing a single patent.
    As a hint - each patent declaration starts with the same line that was
    causing the error found in the previous exercises.
    
    The new files should be saved with filename in the following format:
    "{}-{}".format(filename, n) where n is a counter, starting from 0.
    """
    
    with open(filename,'r') as f:
        lines = f.readlines()
    
    line_num = 0
  
    line_start = 0
    
    n = 0
    
   
    for line in lines[1:]:
        # print lines.index(line)
        line_num = line_num + 1
        if line.find('xml version="1.0"') != -1 or line_num == (len(lines) - 1):
            print line_num
            
            with open("{}-{}".format(PATENTS, n),'w') as f:
                
               for line in  lines[line_start:line_num]:
                    f.write(line)
               n = n + 1
               line_start = line_num
 
   
    pass
   
   

def test():
    split_file(PATENTS)
    for n in range(4):
        try:
            fname = "{}-{}".format(PATENTS, n)
            f = open(fname, "r")
            if not f.readline().startswith("<?xml"):
                print "You have not split the file {} in the correct boundary!".format(fname)
            f.close()
        except:
            print "Could not find file {}. Check if the filename is correct!".format(fname)


# test()

split_file(PATENTS)